In [36]:
import pandas as pd
import numpy as np

from pymongo import MongoClient

import re

#fixed issue with ascii encoding errors
import sys
# reload(sys)
# sys.setdefaultencoding("utf-8")

In [4]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.POETRY

In [5]:
poets = db.POET
poems = db.POEM

In [6]:
rawpoet = pd.DataFrame(list(poets.find()))
rawpoem = pd.DataFrame(list(poems.find()))

In [7]:
# clean rawpoet table.
# at the moment, _id, poet_positions, & posyears (position years) column values appear unusable
# drop these columns for now
ctodrop = ['_id', 'poet_positions', 'poet_posyears']

rawpoet.drop(ctodrop, 1,inplace = True)

In [8]:
rawpoet.head()[:1]

#because this data has been imported straight from mongodb collection, e.g. some columns are actually list 
#types (of ints, or strings), we need to deal with them

,movement_name,movement_url,poet_basicbio,poet_bio,poet_dob2,poet_name,poet_poems_url,poet_tags,poet_url
0,[Slam/Spoken Word],http://www.poets.org/poetsorg/poets?field_scho...,[],[Taylor Mali is a four-time National Poetry Sl...,NaN,[Taylor Mali],http://www.poets.org/poetsorg/poems/406436,"[\n , , \n ...",http://www.poets.org/poetsorg/poet/taylor-mali


In [12]:
ctostr = ['movement_name','poet_bio','poet_name']

for i in ctostr:
    rawpoet[i] = rawpoet[i].map(lambda x: u''.join(x))

In [15]:
# columns poet_tags, poet_basicbio, poet_dob2 are lists
#first convert them to data frames
ptags = rawpoet['poet_tags'].apply(pd.Series)
plife = rawpoet['poet_dob2'].apply(pd.Series)
pbio = rawpoet['poet_basicbio'].apply(pd.Series)

In [138]:
#first lets take care of ptags
# ptags.head()
# ptags.dtypes

#some columns are empty, so drop them
# DONT USE THE CODE BELOW IT IS WRONG
# def isempty(dataframe, column):
#     n = False
#     values = dataframe[column].value_counts()    
#     if len(values) == 1 and len(str(values.index[0]).replace('\n','').replace(' ','').replace('-','').strip()) == 0:
#         n = True        
#     else:
#         c = []
#         for l in range(1,len(values)):
#             if len(c) < 2:
#                 c.append(values.index[l-1])
#             if len(c) > 1:
#                 first = str(c[l-1]).replace('\n','').replace(' ','').replace('-','').strip()
#                 second = str(c[1]).replace('\n','').replace(' ','').replace('-','').strip()
#                 if (len(first) == len(second)) and (first == second):
#                     n = True
#     return values, c


# def coltodrop(dataframe):
#     col = []
#     todrop = []
#     for c in dataframe.columns:
#         col.append(c)
    
#     for c in col:
#         if isempty(dataframe, c):
#             todrop.append(c)
    
#     return todrop
        

ctodrop = [0,1,2]

ptags.drop(ctodrop, axis = 1, inplace = True)

ptags.head()

,3,4,5
0,\n Slam/Spoken Word,NaN,NaN
1,\n Contemporary,NaN,NaN
2,-,,\n Objectivists
3,-,,\n Surrealism
4,-,\n Surrealism,NaN


In [139]:
ptags.columns = ['first', 'second', 'third']
ptags.fillna('', inplace = True)
ptags['first'] = ptags['first'].map(lambda x: x.strip().strip('\n').strip('-'))
ptags['second'] = ptags['second'].map(lambda x: str(x).strip().strip('\n').strip('-'))
ptags['third'] = ptags['third'].map(lambda x: str(x).strip().strip('\n').strip('-'))
ptags[:5]

,first,second,third
0,Slam/Spoken Word,,
1,Contemporary,,
2,,,Objectivists
3,,,Surrealism
4,,Surrealism,


In [142]:
plife.columns = ['yr_born', 'yr_died']

In [143]:
ptags['tags'] = ptags[['first', 'second', 'third']].apply(lambda x: ','.join(x), axis=1)
ptags['tags'] = ptags['tags'].map(lambda x: x.rstrip(',').lstrip(','))
ptags['tags'] = ptags['tags'].map(lambda x: list(x.split(',')))

In [144]:
ptags = ptags['tags'].apply(pd.Series)
ptags.head()

,0,1,2,3
0,Slam/Spoken Word,NaN,NaN,NaN
1,Contemporary,NaN,NaN,NaN
2,Objectivists,NaN,NaN,NaN
3,Surrealism,NaN,NaN,NaN
4,Surrealism,NaN,NaN,NaN


In [149]:
ptags[3].value_counts()

 San Francisco Renaissance    2
 Objectivists                 1
 Surrealism                   1
dtype: int64

In [150]:
ptags.columns = ['tag2', 'tag3', 'tag4','tag5']
rawpoet.drop(['poet_dob2', 'poet_tags'], axis = 1, inplace = True)

In [194]:
#pbio contains everything scraped from poet basic bio section, which includes, secondary yr born, yr died
#, city, state, country, need to parse, clean
# pbio = rawpoet['poet_basicbio'].apply(pd.Series)
pbio.columns = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n']

In [195]:
pbio.a.value_counts()

            165
dtype: int64

In [196]:
pbio.drop(['a','c','k','n'], axis =1, inplace = True)

In [197]:
# pbio['b'].map(lambda x: '' if str(x)=='nan' else str(x))
pbio['a'] = pbio['b'].map(lambda x: '' if str(x)=='nan' else str(x))
pbio['city']= pbio['a'].map(lambda x: '' if x.isdigit() else x)
pbio['yr_born2']= pbio['a'].map(lambda x: x if x.isdigit() else 0)
pbio.drop(['a', 'b'], axis = 1, inplace = True)

In [198]:
pbio['yr_died2']=pbio['d'].map(lambda x: '' if str(x)=='nan' else str(x))
# pbio.yr_died2.value_counts()
pbio['yr_died2']= pbio['yr_died2'].map(lambda x: x if x.isdigit() else 0)

In [199]:
pbio.drop('d', axis = 1, inplace = True)

In [200]:
pbio['state'] = pbio['e'].map(lambda x: str(x) if len(str(x).strip()) == 2 else '')
pbio.e.value_counts()

                   92
                   53
NY                  3
DC                  3
, Cuba              1
PA                  1
TN                  1
MI                  1
TX                  1
OK                  1
, Jamaica           1
AL                  1
FL                  1
, United States     1
CA                  1
VA                  1
dtype: int64

In [201]:
pbio['country'] = pbio['e'].map(lambda x: str(x).strip().replace(',','').replace('nan','') if len(str(x).strip()) != 2 else '')
pbio.drop('e', axis = 1, inplace = True)

In [202]:
# pbio['f'][127:129]
pbio['city2'] = pbio['f'].map(lambda x: str(x).strip().replace(',','').replace('nan','') if len(str(x).strip()) != 2 else '')
pbio['state2']=pbio['f'].map(lambda x: str(x).strip().replace(',','').replace('nan','') if len(str(x).strip()) == 2 else '')

In [203]:
pbio.state2.value_counts()[:10]
pbio.g.value_counts()[:10]

                 92
                 15
St. Louis         3
Baltimore         2
Boston            2
Detroit           2
Newark            2
Richton           1
San Francisco     1
New York City     1
dtype: int64

In [204]:
pbio['city3']= pbio['g'].map(lambda x: str(x).strip().replace('nan',''))

In [205]:
pbio.drop(['f','g'], axis = 1, inplace = True)

In [206]:
pbio.h.value_counts()
pbio['h'] = pbio['h'].map(lambda x: str(x).replace(' ','').replace(',','').replace('nan',''))

In [207]:
pbio['country2'] = pbio['h']

In [208]:
pbio['state3'] = pbio['i'].map(lambda x: str(x).strip().replace(',','').replace('nan','') if len(str(x)) == 2 else '')

In [209]:
pbio['country3']=pbio['i'].map(lambda x: str(x).strip().replace(',','').replace('nan','') if len(str(x)) != 2 else '')

In [210]:
pbio['state4'] = pbio['j'].map(lambda x: str(x).strip().replace(',','').replace('nan','') if len(str(x)) == 2 else '')
pbio['country4']=pbio['j'].map(lambda x: str(x).strip().replace(',','').replace('nan','') if len(str(x)) != 2 else '')

In [211]:
pbio['country5'] = pbio['l'].map(lambda x: str(x).replace(' ','').replace(',','').replace('nan',''))
pbio['country6'] = pbio['m'].map(lambda x: str(x).replace(' ','').replace(',','').replace('nan',''))

In [212]:
pbio.drop(['h','i','j','l','m'], axis = 1, inplace = True)

In [213]:
pbio['country'] = pbio[['country', 'country2', 'country3', 'country4', 'country5', 'country6']].apply(lambda x: ''.join(x), axis=1)

In [214]:
pbio['state'] = pbio[['state', 'state2', 'state3', 'state4']].apply(lambda x: ''.join(x), axis=1)

In [215]:
pbio['city'] = pbio[['city','city2', 'city3']].apply(lambda x: ''.join(x).replace(',',''), axis=1)

In [216]:
pbio.drop(['city2', 'city3','state2', 'state3', 'state4','country2', 'country3', 'country4', 'country5', 'country6'], axis = 1, inplace = True)

In [217]:
pbio.head()

,city,yr_born2,yr_died2,state,country
0,,0,0,,
1,,0,0,,
2,Cincinnati,1942,1997,,United States
3,,0,0,,
4,,0,0,,


In [219]:
poetdf = pd.concat([rawpoet,plife,ptags,pbio], axis=1)

In [220]:
poetdf.drop('poet_basicbio', axis = 1, inplace = True)

In [221]:
poetdf.head()

,movement_name,movement_url,poet_bio,poet_name,poet_poems_url,poet_url,yr_born,yr_died,tag2,tag3,tag4,tag5,city,yr_born2,yr_died2,state,country
0,Slam/Spoken Word,http://www.poets.org/poetsorg/poets?field_scho...,Taylor Mali is a four-time National Poetry Sla...,Taylor Mali,http://www.poets.org/poetsorg/poems/406436,http://www.poets.org/poetsorg/poet/taylor-mali,NaN,NaN,Slam/Spoken Word,NaN,NaN,NaN,,0,0,,
1,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,Richard Garcia is the author of five books of ...,Richard Garcia,http://www.poets.org/poetsorg/poems/45709,http://www.poets.org/poetsorg/poet/richard-garcia,NaN,NaN,Contemporary,NaN,NaN,NaN,,0,0,,
2,Objectivists,http://www.poets.org/poetsorg/poets?field_scho...,"Born in Cincinnati, Ohio, on November 11, 1942...",William Matthews,http://www.poets.org/poetsorg/poems/45636,http://www.poets.org/poetsorg/poet/william-mat...,1942,1997,Objectivists,NaN,NaN,NaN,Cincinnati,1942,1997,,United States
3,Surrealism,http://www.poets.org/poetsorg/poets?field_scho...,Archie Randolph Ammons was born outside Whitev...,A. R. Ammons,http://www.poets.org/poetsorg/poems/44389,http://www.poets.org/poetsorg/poet/r-ammons,1926,2001,Surrealism,NaN,NaN,NaN,,0,0,,
4,Surrealism,http://www.poets.org/poetsorg/poets?field_scho...,"Born in Indiana on February 8, 1926, Philip Ap...",Philip Appleman,http://www.poets.org/poetsorg/poems/46510,http://www.poets.org/poetsorg/poet/philip-appl...,1926,NaN,Surrealism,NaN,NaN,NaN,,0,0,,


In [223]:
poetdf.fillna(0, inplace = True)

,movement_name,movement_url,poet_bio,poet_name,poet_poems_url,poet_url,yr_born,yr_died,tag2,tag3,tag4,tag5,city,yr_born2,yr_died2,state,country
0,Slam/Spoken Word,http://www.poets.org/poetsorg/poets?field_scho...,Taylor Mali is a four-time National Poetry Sla...,Taylor Mali,http://www.poets.org/poetsorg/poems/406436,http://www.poets.org/poetsorg/poet/taylor-mali,0,0,Slam/Spoken Word,0,0,0,,0,0,,
1,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,Richard Garcia is the author of five books of ...,Richard Garcia,http://www.poets.org/poetsorg/poems/45709,http://www.poets.org/poetsorg/poet/richard-garcia,0,0,Contemporary,0,0,0,,0,0,,
2,Objectivists,http://www.poets.org/poetsorg/poets?field_scho...,"Born in Cincinnati, Ohio, on November 11, 1942...",William Matthews,http://www.poets.org/poetsorg/poems/45636,http://www.poets.org/poetsorg/poet/william-mat...,1942,1997,Objectivists,0,0,0,Cincinnati,1942,1997,,United States
3,Surrealism,http://www.poets.org/poetsorg/poets?field_scho...,Archie Randolph Ammons was born outside Whitev...,A. R. Ammons,http://www.poets.org/poetsorg/poems/44389,http://www.poets.org/poetsorg/poet/r-ammons,1926,2001,Surrealism,0,0,0,,0,0,,
4,Surrealism,http://www.poets.org/poetsorg/poets?field_scho...,"Born in Indiana on February 8, 1926, Philip Ap...",Philip Appleman,http://www.poets.org/poetsorg/poems/46510,http://www.poets.org/poetsorg/poet/philip-appl...,1926,0,Surrealism,0,0,0,,0,0,,


In [226]:
poetdf[['yr_born', 'yr_died','yr_born2', 'yr_died2']] = poetdf[['yr_born', 'yr_died','yr_born2', 'yr_died2']].astype(int)

In [237]:
poetdf.loc[poetdf['yr_died']>0,:]

poetdf['dead'] = np.where(poetdf['yr_died'] > 0,1,0)
poetdf['current_age'] = np.where(poetdf['dead'] == 0, (2015- poetdf['yr_born']), 0)
poetdf['age_dead'] = np.where(poetdf['dead'] == 1, (poetdf['yr_died']-poetdf['yr_born']), 0)

In [238]:
poetdf['current_age'] = np.where(poetdf['current_age'] == 2015, 0,poetdf['current_age'])

In [243]:
poetdf['tag2'] = np.where(poetdf['tag2'] == 0, '',poetdf['tag2'])
poetdf['tag3'] = np.where(poetdf['tag3'] == 0, '',poetdf['tag3'])
poetdf['tag4'] = np.where(poetdf['tag4'] == 0, '',poetdf['tag4'])
poetdf['tag5'] = np.where(poetdf['tag5'] == 0, '',poetdf['tag5'])

In [245]:
poetdf.loc[poetdf['tag2']<>poetdf['movement_name'],:]

,movement_name,movement_url,poet_bio,poet_name,poet_poems_url,poet_url,yr_born,yr_died,tag2,tag3,tag4,tag5,city,yr_born2,yr_died2,state,country,dead,current_age,age_dead
6,Surrealism,http://www.poets.org/poetsorg/poets?field_scho...,"Aimé Césaire was born June 26, 1913, in Basse-...",Aimé Césaire,http://www.poets.org/poetsorg/poems/45700,http://www.poets.org/poetsorg/poet/aim%C3%A9-c...,1913,2008,Negritude,Surrealism,,,,0,0,,,1,0,95
100,Modernism,http://www.poets.org/poetsorg/poets?field_scho...,"In 1921 Ezra Pound wrote to Marianne Moore: ""I...",Mina Loy,http://www.poets.org/poetsorg/poems/45476,http://www.poets.org/poetsorg/poet/mina-loy,1882,1966,Fireside Poet,Jazz Poetry,Modernism,,London,1882,1966,,United Kingdom,1,0,84
106,Modernism,http://www.poets.org/poetsorg/poets?field_scho...,"read this poet's poemsOn September 17, 1883, W...",William Carlos Williams,http://www.poets.org/poetsorg/poems/45484,http://www.poets.org/poetsorg/poet/william-car...,1883,1963,Imagism,Metaphysical Poet,Modernism,Objectivists,Rutherford,1883,1963,NJ,UnitedStates,1,0,80
112,Dark Room Collective,http://www.poets.org/poetsorg/poets?field_scho...,"Natasha Trethewey was born on April 26, 1966, ...",Natasha Trethewey,http://www.poets.org/poetsorg/poems/46066,http://www.poets.org/poetsorg/poet/natasha-tre...,1966,0,Contemporary,Dark Room Collective,,,,0,0,,,0,49,0
138,Confessional Poetry,http://www.poets.org/poetsorg/poets?field_scho...,"Mark Doty was born on August 10, 1953. He is t...",Mark Doty,http://www.poets.org/poetsorg/poems/81376,http://www.poets.org/poetsorg/poet/mark-doty,1953,0,Concrete Poetry,Confessional Poetry,,,Maryville,1953,0,TN,UnitedStates,0,62,0
165,Symbolists,http://www.poets.org/poetsorg/poets?field_scho...,The son of Joseph-Francois Baudelaire and Caro...,Charles Baudelaire,http://www.poets.org/poetsorg/poems/45795,http://www.poets.org/poetsorg/poet/charles-bau...,1821,1867,Surrealism,Symbolists,,,,0,0,,,1,0,46
172,Modernism,http://www.poets.org/poetsorg/poets?field_scho...,"On September 10, 1886, Hilda Doolittle was bor...",H. D.,http://www.poets.org/poetsorg/poems/45778,http://www.poets.org/poetsorg/poet/h-d,1886,1961,Imagism,Metaphysical Poet,Modernism,,Bethlehem,1886,1961,PA,UnitedStates,1,0,75
325,Symbolists,http://www.poets.org/poetsorg/poets?field_scho...,"A volatile and peripatetic poet, the prodigy A...",Arthur Rimbaud,http://www.poets.org/poetsorg/poems/44836,http://www.poets.org/poetsorg/poet/arthur-rimbaud,1854,1891,Surrealism,Symbolists,,,,0,0,,,1,0,37
330,Jazz Poetry,http://www.poets.org/poetsorg/poets?field_scho...,read poems by this poetJames Mercer Langston H...,Langston Hughes,http://www.poets.org/poetsorg/poems/44733,http://www.poets.org/poetsorg/poet/langston-hu...,1902,1967,Harlem Renaissance,Jazz Poetry,,,Joplin,1902,1967,MO,UnitedStates,1,0,65
335,Modernism,http://www.poets.org/poetsorg/poets?field_scho...,"Wallace Stevens was born in Reading, Pennsylva...",Wallace Stevens,http://www.poets.org/poetsorg/poems/45494,http://www.poets.org/poetsorg/poet/wallace-ste...,1879,1955,Metaphysical Poet,Modernism,,,Reading,1879,1955,PA,UnitedStates,1,0,76


In [266]:
poetdf[(poetdf['movement_name'] == 'Contemporary')].sort('tag3', ascending = False)

,movement_name,movement_url,poet_bio,poet_name,poet_poems_url,poet_url,yr_born,yr_died,tag2,tag3,tag4,tag5,city,yr_born2,yr_died2,state,country,dead,current_age,age_dead
63,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,"Edwin Torres is a bilingual, New York-based po...",Edwin Torres,http://www.poets.org/poetsorg/poems/45374,http://www.poets.org/poetsorg/poet/edwin-torres,0,0,Contemporary,Slam/Spoken Word,,,,0,0,,,0,0,0
317,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,"Born on June 25, 1955, Patricia Smith is a poe...",Patricia Smith,http://www.poets.org/poetsorg/poems/44388,http://www.poets.org/poetsorg/poet/patricia-smith,1955,0,Contemporary,Slam/Spoken Word,,,Chicago,1955,0,IL,UnitedStates,0,60,0
26,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,Todd Boss was born in 1968 and raised on a cat...,Todd Boss,http://www.poets.org/poetsorg/poems/46776,http://www.poets.org/poetsorg/poet/todd-boss,0,0,Contemporary,,,,,0,0,,,0,0,0
27,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,Ana Božičević is the author of Rise in the Fal...,Ana Božičević,http://www.poets.org/poetsorg/poems/45010,http://www.poets.org/poetsorg/poet/ana-bo%C5%B...,0,0,Contemporary,,,,,0,0,,,0,0,0
28,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,"Born in Detroit, Michigan in 1954, Catherine A...",Catherine Anderson,http://www.poets.org/poetsorg/poems/44664,http://www.poets.org/poetsorg/poet/catherine-a...,0,0,Contemporary,,,,Detroit,0,0,MI,UnitedStates,0,0,0
29,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,"Ralph Angel was born in Seattle, Washington, i...",Ralph Angel,http://www.poets.org/poetsorg/poems/45502,http://www.poets.org/poetsorg/poet/ralph-angel,0,0,Contemporary,,,,,0,0,,,0,0,0
30,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,Jonathan Aaron is the author of the poetry col...,Jonathan Aaron,http://www.poets.org/poetsorg/poems/45361,http://www.poets.org/poetsorg/poet/jonathan-aaron,0,0,Contemporary,,,,,0,0,,,0,0,0
31,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,Chris Abani is the author of Sanctificum (Copp...,Chris Abani,http://www.poets.org/poetsorg/poems/45407,http://www.poets.org/poetsorg/poet/chris-abani,0,0,Contemporary,,,,,0,0,,,0,0,0
32,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,Seth Abramson is the author of the poetry coll...,Seth Abramson,http://www.poets.org/poetsorg/poems/44879,http://www.poets.org/poetsorg/poet/seth-abramson,0,0,Contemporary,,,,,0,0,,,0,0,0
33,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,A recipient of the Academy of American Poets L...,Diane Ackerman,http://www.poets.org/poetsorg/poems/46318,http://www.poets.org/poetsorg/poet/diane-ackerman,0,0,Contemporary,,,,,0,0,,,0,0,0


In [268]:
poetdf.movement_name.value_counts()

Contemporary                 202
Modernism                     15
Formalism                     14
Language Poetry               13
Surrealism                    12
Confessional Poetry           11
New York School               11
Beat                          11
Black Arts                     9
Symbolists                     8
Jazz Poetry                    7
Harlem Renaissance             7
Conceptual Poetry              6
Objectivists                   6
Black Mountain                 4
Dark Room Collective           4
Concrete Poetry                3
San Francisco Renaissance      3
Slam/Spoken Word               2
New Formalism                  1
dtype: int64

In [271]:
rawpoem.head()
# rawpoet[i] = rawpoet[i].map(lambda x: u''.join(x))
poemdf = rawpoem[['poet_poems_url','poem_title', 'poem_yrpub', 'poem_text']]

In [273]:
resultdf = pd.merge(poetdf, poemdf, how='inner', on=['poet_poems_url'])

In [278]:
resultdf.head()

,movement_name,movement_url,poet_bio,poet_name,poet_poems_url,poet_url,yr_born,yr_died,tag2,tag3,...,yr_born2,yr_died2,state,country,dead,current_age,age_dead,poem_title,poem_yrpub,poem_text
0,Slam/Spoken Word,http://www.poets.org/poetsorg/poets?field_scho...,Taylor Mali is a four-time National Poetry Sla...,Taylor Mali,http://www.poets.org/poetsorg/poems/406436,http://www.poets.org/poetsorg/poet/taylor-mali,0,0,Slam/Spoken Word,,...,0,0,,,0,0,0,My Deepest Condiments,2014,"[I send you my deepest condiments, was in no w..."
1,Contemporary,http://www.poets.org/poetsorg/poets?field_scho...,Richard Garcia is the author of five books of ...,Richard Garcia,http://www.poets.org/poetsorg/poems/45709,http://www.poets.org/poetsorg/poet/richard-garcia,0,0,Contemporary,,...,0,0,,,0,0,0,Autumn,2006,"[\nBoth lying on our sides, making love in\nsp..."
2,Objectivists,http://www.poets.org/poetsorg/poets?field_scho...,"Born in Cincinnati, Ohio, on November 11, 1942...",William Matthews,http://www.poets.org/poetsorg/poems/45636,http://www.poets.org/poetsorg/poet/william-mat...,1942,1997,Objectivists,,...,1942,1997,,United States,1,0,55,The Bear at the Dump,1995,[\nAmidst the too much that we buy and throw\n...
3,Surrealism,http://www.poets.org/poetsorg/poets?field_scho...,Archie Randolph Ammons was born outside Whitev...,A. R. Ammons,http://www.poets.org/poetsorg/poems/44389,http://www.poets.org/poetsorg/poet/r-ammons,1926,2001,Surrealism,,...,0,0,,,1,0,75,In Memoriam Mae Noblitt,1981,"[\nThis is just a place:\nwe go around, distan..."
4,Surrealism,http://www.poets.org/poetsorg/poets?field_scho...,Archie Randolph Ammons was born outside Whitev...,A. R. Ammons,http://www.poets.org/poetsorg/poems/44389,http://www.poets.org/poetsorg/poet/r-ammons,1926,2001,Surrealism,,...,0,0,,,1,0,75,Still,1986,[\nI said I will find what is lowly\nand put t...
